##### Copyright 2022 Die TensorFlow-Autoren.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Komponieren von Lernalgorithmen

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/composing_learning_algorithms"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">VIEN auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.by/github/tensorflow/federated/blob/v0.33.0/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">In Google Colab ausführen</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.3.0/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notebofrok herunterladen</a></td>
</table>

## Bevor du anfängst

Bevor Sie beginnen, führen Sie bitte Folgendes aus, um sicherzustellen, dass Ihre Umgebung richtig eingerichtet ist. Wenn Sie keine Begrüßung sehen, finden Sie Anweisungen in der [Installationsanleitung](../install.md) . 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from typing import Callable

import tensorflow as tf
import tensorflow_federated as tff

**HINWEIS** : Es wurde verifiziert, dass diese Zusammenarbeit mit der [neuesten veröffentlichten Version](https://github.com/tensorflow/federated#compatibility) des pip-Pakets `tensorflow_federated` , aber das Tensorflow Federated-Projekt befindet sich noch in der Entwicklung vor der Veröffentlichung und funktioniert möglicherweise nicht auf `main` .

# Komponieren von Lernalgorithmen

Das Lernprogramm „ [Building Your Own Federated Learning Algorithm](https://github.com/tensorflow/federated/blob/v0.33.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) “ verwendete den Federated Core von TFF, um direkt eine Version des Federated Averaging (FedAvg)-Algorithmus zu implementieren.

In diesem Lernprogramm verwenden Sie föderierte Lernkomponenten in der TFF-API, um föderierte Lernalgorithmen auf modulare Weise zu erstellen, ohne alles von Grund auf neu implementieren zu müssen.

Für die Zwecke dieses Lernprogramms implementieren Sie eine Variante von FedAvg, die das Gradienten-Clipping durch lokales Training verwendet.

## Lernalgorithmus-Bausteine

Auf hoher Ebene können viele Lernalgorithmen in 4 separate Komponenten unterteilt werden, die als **Bausteine** bezeichnet werden. Diese sind wie folgt:

1. Distributor (d. h. Server-zu-Client-Kommunikation)
2. Client-Arbeit (dh lokale Client-Berechnung)
3. Aggregator (d. h. Client-zu-Server-Kommunikation)
4. Finalizer (d. h. Serverberechnung mit aggregierten Clientausgaben)

Während das [Lernprogramm „Building Your Own Federated Learning Algorithm“](https://github.com/tensorflow/federated/blob/v0.33.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) all diese Bausteine von Grund auf implementiert hat, ist dies oft unnötig. Stattdessen können Sie Bausteine aus ähnlichen Algorithmen wiederverwenden.

In diesem Fall müssen Sie zum Implementieren von FedAvg mit Verlaufsbeschneidung nur den **Clientarbeitsbaustein** ändern. Die verbleibenden Blöcke können mit denen identisch sein, die in "Vanilla" FedAvg verwendet werden.

# Umsetzung der Kundenarbeit

Lassen Sie uns zunächst eine TF-Logik schreiben, die ein lokales Modelltraining mit Gradientenbeschneidung durchführt. Der Einfachheit halber werden Farbverläufe mit einer Norm von höchstens 1 beschnitten.

## TF-Logik

In [ ]:
@tf.function
def client_update(model: tff.learning.Model,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

Es gibt ein paar wichtige Punkte zum obigen Code. Erstens verfolgt es die Anzahl der gesehenen Beispiele, da dies die *Gewichtung* der Client-Aktualisierung darstellt (wenn ein Durchschnitt über Clients berechnet wird).

Zweitens verwendet es [`tff.learning.templates.ClientResult`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientResult) , um die Ausgabe zu packen. Dieser Rückgabetyp wird verwendet, um Client-Arbeitsbausteine in `tff.learning` zu standardisieren.

## Erstellen eines ClientWorkProcess

Während die obige TF-Logik ein lokales Training mit Clipping durchführt, muss sie dennoch in TFF-Code verpackt werden, um den erforderlichen Baustein zu erstellen.

Insbesondere werden die 4 Bausteine als [`tff.templates.MeasuredProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/MeasuredProcess) . Dies bedeutet, dass alle 4 Blöcke sowohl eine `initialize` als auch eine `next` -Funktion haben, die verwendet werden, um die Berechnung zu instanziieren und auszuführen.

Dadurch kann jeder Baustein seinen eigenen (auf dem Server gespeicherten) **Zustand** nachverfolgen, wie er für die Durchführung seiner Operationen benötigt wird. Obwohl es in diesem Lernprogramm nicht verwendet wird, kann es für Dinge wie das Verfolgen der Anzahl der aufgetretenen Iterationen oder das Verfolgen von Optimiererzuständen verwendet werden.

Clientarbeits-TF-Logik sollte im Allgemeinen als [`tff.learning.templates.ClientWorkProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientWorkProcess) werden, das die erwarteten Typen kodiert, die in das lokale Training des Clients ein- und ausgehen. Es kann wie unten durch ein Modell und einen Optimierer parametrisiert werden.

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.Model],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.framework.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# Erstellen eines Lernalgorithmus

Lassen Sie uns die obige Client-Arbeit in einen vollwertigen Algorithmus packen. Zuerst richten wir unsere Daten und unser Modell ein.

## Aufbereitung der Eingabedaten

Laden Sie den in TFF enthaltenen EMNIST-Datensatz und verarbeiten Sie ihn vor. Weitere Einzelheiten finden Sie im Lernprogramm zur [Bildklassifizierung](federated_learning_for_image_classification.ipynb) .

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Um den Datensatz in unser Modell einzuspeisen, werden die Daten geglättet und in Tupel der Form `(flattened_image_vector, label)` umgewandelt.

Wählen wir eine kleine Anzahl von Kunden aus und wenden die obige Vorverarbeitung auf ihre Datensätze an.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Vorbereiten des Modells

Dabei wird dasselbe Modell wie im Lernprogramm zur [Bildklassifizierung](federated_learning_for_image_classification.ipynb) verwendet. Dieses Modell (implementiert über `tf.keras` ) hat eine einzelne verborgene Schicht, gefolgt von einer Softmax-Schicht. Um dieses Modell in TFF zu verwenden, wird das Keras-Modell als [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) . Dies ermöglicht es uns, den [Vorwärtsdurchgang](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) des Modells innerhalb von TFF durchzuführen und [Modellausgaben zu extrahieren](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics) . Weitere Einzelheiten finden Sie auch im Tutorial zur [Bildklassifizierung](federated_learning_for_image_classification.ipynb) .

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Vorbereitung der Optimierer

Genau wie in [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) gibt es hier zwei Optimierer: einen Client-Optimierer und einen Server-Optimierer. Der Einfachheit halber sind die Optimierer SGD mit unterschiedlichen Lernraten.

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## Bausteine definieren

Nachdem der Client-Work-Baustein, die Daten, das Modell und die Optimierer eingerichtet sind, müssen noch Bausteine für den Verteiler, den Aggregator und den Finalisierer erstellt werden. Dies kann einfach durch Ausleihen einiger Standardwerte erfolgen, die in TFF verfügbar sind und von FedAvg verwendet werden.

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## Zusammenstellen der Bausteine

Schließlich können Sie einen eingebauten **Composer** in TFF verwenden, um die Bausteine zusammenzusetzen. Dies ist ein relativ einfacher Composer, der die 4 oben genannten Bausteine nimmt und ihre Typen miteinander verbindet.

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# Ausführen des Algorithmus

Nun, da der Algorithmus fertig ist, lassen Sie ihn ausführen. **Initialisieren Sie zunächst** den Algorithmus. Der **Zustand** dieses Algorithmus hat eine Komponente für jeden Baustein, zusammen mit einer für die *globalen Modellgewichte* .

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

Wie erwartet hat die Client-Arbeit einen leeren Zustand (denken Sie an den Client-Arbeitscode oben!). Andere Bausteine können jedoch einen nicht leeren Zustand haben. Beispielsweise verfolgt der Finalizer, wie viele Iterationen stattgefunden haben. Da `next` noch nicht ausgeführt wurde, hat es den Status `0` .

In [ ]:
state.finalizer

[0]

Führen Sie nun eine Trainingsrunde durch.

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

Die Ausgabe davon ( `tff.learning.templates.LearningProcessOutput` ) hat sowohl eine `.state` als auch eine `.metrics` Ausgabe. Schauen wir uns beide an.

In [ ]:
learning_process_output.state.finalizer

[1]

Der Finalizer-Zustand hat sich eindeutig um eins erhöht, da eine Runde von `.next` ausgeführt wurde.

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

Während die Metriken leer sind, sind sie für komplexere und praktischere Algorithmen im Allgemeinen voller nützlicher Informationen.

# Fazit

Indem Sie das obige Baustein-/Composer-Framework verwenden, können Sie völlig neue Lernalgorithmen erstellen, ohne alles von Grund auf neu machen zu müssen. Dies ist jedoch nur der Ausgangspunkt. Dieses Framework macht es viel einfacher, Algorithmen als einfache Modifikationen von FedAvg auszudrücken. Weitere Algorithmen finden Sie unter [`tff.learning.algorithms`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms) , das Algorithmen wie [FedProx](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) und [FedAvg mit Client-Lernratenplanung](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule) enthält. Diese APIs können sogar die Implementierung völlig neuer Algorithmen unterstützen, z. B. [föderiertes k-Means-Clustering](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans) .